In [11]:
# DEPENDENCIES
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Addition needed for statistical analysis
import scipy.stats as st


In [12]:
import numpy as np
import pandas as pd
import datetime as dt


# Reflect Tables into SQLAlchemy ORM

In [13]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# Addition inspect
from sqlalchemy import create_engine, func, inspect

In [14]:
# create engine to hawaii.sqlite
# engine = create_engine("sqlite:///hawaii.sqlite")
engine = create_engine('sqlite:///hawaii_clean.sqlite')

In [15]:
# reflect an existing database into a new model
# Declare a Base using `automap_base()`
Base = automap_base()

# reflect the tables
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [16]:
# View all of the classes that automap found
Base.classes.keys()

[]

In [17]:
# Session link to check columns with inspector
inspector = inspect(engine)


In [18]:
# --- check columns
measures_columns = inspector.get_columns('measurement')
for column in measures_columns:
    print(column["name"], column["type"])

station TEXT
date TEXT
prcp FLOAT
tobs BIGINT


In [19]:
# --- check columns
stations_columns = inspector.get_columns('station')
for column in stations_columns:
    print(column["name"], column["type"])

station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [20]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

AttributeError: measurement

In [21]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [22]:
# Check records
for row in session.query(measurement.id, measurement.station).limit(10).all():
    print(row)

NameError: name 'measurement' is not defined

In [23]:
# check measurements table content and fields
first_row = session.query(measurement).first()
first_row.__dict__

NameError: name 'measurement' is not defined

In [24]:
# check station table content and fields
first_row = session.query(station).first()
first_row.__dict__

NameError: name 'station' is not defined

# Exploratory Precipitation Analysis

In [25]:
# Find the most recent date in the data set.

# --- query the db and get the last data point (first in the descending order list) and find the date ---
most_recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()[0]
most_recent_date


NameError: name 'measurement' is not defined

In [26]:

# # --- convert string date to date format for use with datetime library ---
most_recent_date = (dt.datetime.strptime(most_recent_date, "%Y-%m-%d")).date()
most_recent_date

NameError: name 'most_recent_date' is not defined

In [27]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 

# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
query_date = most_recent_date - dt.timedelta(days=365)
# query_date

# Perform a query to retrieve the data and precipitation scores
precip = session.query(measurement.date, measurement.prcp).filter(measurement.date >= query_date).\
       group_by(measurement.date).all()
precip

NameError: name 'most_recent_date' is not defined

In [28]:
# Save the query results as a Pandas DataFrame
precip_df = pd.DataFrame(precip)
precip_df.head()

NameError: name 'precip' is not defined

In [29]:
#  set the index to the date column
precip_df = precip_df.set_index("date")
precip_df.head()

NameError: name 'precip_df' is not defined

In [30]:
# Sort the dataframe by date.
precip_df = precip_df.sort_values(by=['date'])
precip_df.head()

NameError: name 'precip_df' is not defined

In [31]:
# Use Pandas Plotting with Matplotlib to plot the data

# BEFORE set figure size to adjust for space
precip_df.plot(figsize=(30,10), fontsize =30, legend=None)

#Set plot title.
# plt.title("Precipitation during sample period", fontsize =50)
plt.title(f"Precipitation from {query_date} to {most_recent_date}", fontsize = 50)
plt.xlabel('Dates', fontsize =40, color ='b')
plt.ylabel("Precipitation (inches)", fontsize =40, color ='b')
# set font and rotation for date tick labels
plt.xticks(rotation=45, fontsize =25)


# ax = precip_df.plot()
# # set monthly locator
# ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
# # set formatter
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))

# plt.gcf().autofmt_xdate()

#Save graph. bbox documentation cuts off
plt.savefig("images/Precipitation", bbox_inches='tight')

#Show graph.

plt.show()


NameError: name 'precip_df' is not defined

In [32]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip_df.describe()

NameError: name 'precip_df' is not defined

# Exploratory Station Analysis

In [33]:
# Design a query to calculate the total number stations in the dataset
stations_count = session.query(station).count()

print(f"There are {stations_count} stations in this sample")


NameError: name 'station' is not defined

In [34]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
activity_by_station = session.query(measurement.station, func.count(measurement.station)).\
order_by(func.count(measurement.station).desc()).group_by(measurement.station).all()
activity_by_station


NameError: name 'measurement' is not defined

In [35]:
# most mactive one
most_active = activity_by_station[0]
most_active
print(f"The most active station in this sample is {activity_by_station[0][0]}")
print(f"with a total of {activity_by_station[0][1]} counts")

NameError: name 'activity_by_station' is not defined

In [36]:
# Using the most active station id from the previous query, 
most_active_station = activity_by_station[0][0]

# calculate the lowest or minimum temperature (in most active station)
lowest_temp = session.query(func.min(measurement.tobs)).filter(measurement.station == most_active_station).scalar()
              
print(f"Lowest temperature: {lowest_temp} degrees F.")

# highest or maximum temperature (in most active station)
highest_temp = session.query(func.max(measurement.tobs)).filter(measurement.station == most_active_station).scalar()

print(f"Highest temperature: {highest_temp} degrees F.")

#and average temperature (in most active station)
avg_temp = session.query(func.avg(measurement.tobs)).filter(measurement.station == most_active_station).scalar()

print(f"Average temperature: {round(avg_temp, 2)} degrees F.")

NameError: name 'activity_by_station' is not defined

In [37]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
most_active_temp_df = pd.DataFrame(session.query(measurement.tobs).filter((measurement.station == most_active_station)\
                                        & (measurement.date >= query_date)\
                                        & (measurement.date <= most_recent_date)).all())

most_active_temp_df.plot(kind="hist", figsize=(20,10), fontsize =30, legend=None)

plt.xlabel("Temperature in degrees Farenheit", fontsize=30, color='r')
plt.ylabel("Observation frequency", fontsize=30, color='r')

plt.xticks(fontsize=25)
plt.yticks(fontsize=25)

plt.title(f"Temperature observations @ {most_active_station}", fontsize=30)

# plt.tight_layout()
plt.savefig("images/temperature observations.png", bbox_inches='tight')
plt.show()

NameError: name 'measurement' is not defined

# Close session

In [38]:
# Close Session
session.close()